In [19]:
import pandas as pd
import numpy as np

folder = '../trictrac_database'

In [20]:
# avis_clean = pd.read_csv(f'{folder}/avis_clean.csv', header=None, names=["Game id", "User id", "Game name UI", "Username", "Datetime", "Rating", "Comment title", "Comment body"]).drop_duplicates()
# users = pd.read_csv(f'{folder}/users.csv', header=None, names = ["Username", "User id"]).drop_duplicates()
# avis_clean["Datetime"] = pd.to_datetime(avis_clean["Datetime"]) # convert from object to datetime
jeux_clean = pd.read_csv(f'{folder}/jeux_clean.csv', header=None, names = ["Game id", "Game name website", "Game name UI", "Game name year", "Description", "Type", "Extra"])
jeux_clean[jeux_clean["Game name website"] == 'cave-evil']


,Game id,Game name website,Game name UI,Game name year,Description,Type,Extra
1598,1598,cave-evil,Cave Evil,Cave Evil (2011),Aucune description,NaN,2 à 4|14 à 99 |60240


In [21]:

# Separate extra info (number of player, age categories, game duration), replace ~ by NaN
jeux_clean[["Number of players", "Age", "Game duration"]] = jeux_clean.pop("Extra").str.split('|',n=3, expand=True, regex=False).replace('~', np.nan)
jeux_clean[jeux_clean["Game id"] == 99]

,Game id,Game name website,Game name UI,Game name year,Description,Type,Number of players,Age,Game duration
99,99,6-qui-prend-anniversaire,6 Qui prend ! anniversaire,6 Qui prend ! anniversaire (2014),Ces drôles de cartes valent de 1 à 7 «têtes de...,Ambiance|Bluff,2 à 10,10 ans et +,30


In [38]:
# Number of players separation
jeux_clean["Number of players"] = jeux_clean["Number of players"].replace(r"jusqu\'à (\d+)", r"1 à \1", regex=True) # 'jusqu'à num' to '1 à num'
jeux_clean["Number of players"] = jeux_clean["Number of players"].replace(r"à partir de (\d+)", r"\1 à 0", regex=True) # 'à partir de num' to 'num à 0'
jeux_clean["Number of players"] = jeux_clean["Number of players"].replace(r"^\s*(\d+)\s*$", r"\1 à \1", regex=True) # 'num' to 'num à num'

jeux_clean[["Min number of players", "Max number of players"]] = jeux_clean.pop("Number of players").str.extract(r"(\d+)\s*à\s*(\d+)")
# Set types
jeux_clean["Min number of players"] = jeux_clean["Min number of players"].astype('Int16')
jeux_clean["Max number of players"] = jeux_clean["Max number of players"].astype('Int16')
jeux_clean["Max number of players"] = jeux_clean["Max number of players"].replace(0, np.nan,  regex=False)
jeux_clean[jeux_clean["Game id"] == 99]

KeyError: 'Number of players'

In [40]:
jeux_clean[(jeux_clean["Min number of players"] > 10) | (jeux_clean["Max number of players"] > 10)]

,Game id,Game name website,Game name UI,Game name year,Description,Type,Game duration,Min number of players,Max number of players,Age min,Age max
31,31,15-indices-sur-150-choses-lieux-personnages,15 indices sur 150 choses lieux personnages,15 indices sur 150 choses lieux personnages (1...,"""15 indices"" est similaire à son successeur ""I...",Jeux de connaissances,15,2,12,12,99
158,158,a-la-votre-2,A la vôtre !,A la vôtre ! (2017),"Dans l'ancien royaume d'Otravia, tout le monde...",NaN,30,2,12,14,99
159,159,a-touch-of-evil-the-coast,A touch of Evil - The Coast,A touch of Evil - The Coast (2012),The coast est une nouvelle extension pour le j...,Jeux de plateau,120,2,12,12,99
232,232,activity-0,Activity,Activity (2014),Le jeu d'ambiance de Piatnik! Activity est un ...,NaN,45,3,16,12,99
301,301,agents-secrets-0,Agents Secrets,Agents Secrets (2014),"Seconde édition du jeu Agents Secrets, initial...",Affrontement|Ambiance|Bluff|Choix simultanés|E...,30,6,12,10,99
...,...,...,...,...,...,...,...,...,...,...,...
10153,10153,win,Win !,Win ! (2012),Chaque joueur récupère une carte de vote recto...,Vote|Ambiance|Humour,30,5,11,10,99
10218,10218,world-magic-park-3d,World Magic Park 3D,World Magic Park 3D (2003),«World Magic Park» est un jeu multi univers av...,Jeux de cartes,<NA>,1,100,10,99
10222,10222,world-cup-tournament-football-game,World cup Tournament Football Game,World cup Tournament Football Game (1993),World Cup est une simulation d'un championnat ...,Jeux de cartes,60,2,24,6,99
10506,10506,a-l-heure-du-crime-ou-etiez-vous,À l'heure du crime... où étiez-vous ?,À l'heure du crime... où étiez-vous ? (2009),À l'heure du crime est un jeu d'ambiance rapid...,NaN,20,5,20,8,99


In [24]:
# Age sepration
jeux_clean["Age"] = jeux_clean["Age"].replace(r"(\d+) ans et \+", r"\1 à 99", regex=True) # max age is fixed to 99
jeux_clean[["Age min", "Age max"]] = jeux_clean.pop("Age").str.extract(r"(\d+) à (\d+)")

In [25]:
# Type conversion
jeux_clean["Age max"] = jeux_clean["Age max"].astype('Int64')
jeux_clean["Age min"] = jeux_clean["Age min"].astype('Int64')
jeux_clean["Game duration"] = jeux_clean["Game duration"].astype('Int64')
jeux_clean.loc[jeux_clean["Age max"] > 99, "Age max"] = 99
jeux_clean[jeux_clean["Game id"] == 99]

,Game id,Game name website,Game name UI,Game name year,Description,Type,Game duration,Min number of players,Max number of players,Age min,Age max
99,99,6-qui-prend-anniversaire,6 Qui prend ! anniversaire,6 Qui prend ! anniversaire (2014),Ces drôles de cartes valent de 1 à 7 «têtes de...,Ambiance|Bluff,30,2,10,10,99


In [26]:
print(jeux_clean.shape)
jeux_clean.drop_duplicates()


(10549, 11)


,Game id,Game name website,Game name UI,Game name year,Description,Type,Game duration,Min number of players,Max number of players,Age min,Age max
0,0,fever,$ Fever,$ Fever (1990),Le jeu comprend 6 dés dont les faces représent...,Jeux de dés,20,2,6,8,99
1,1,cetera,&cetera,&cetera (2003),&cetera est une extension de Roads & Boats qui...,Jeux de plateau,<NA>,1,6,14,99
2,2,43,'43,'43 (1981),'43 simule le débarquement en Sicile des force...,Jeux de guerre,240,2,4,12,99
3,3,und-tschuss,...Und Tschüss,...Und Tschüss (1997),But du jeu Remporter les cartes les plus forte...,Jeux de cartes,30,4,6,10,99
4,4,011,011,011 (2011),"L’histoire... Nous sommes en l’an 011, au XIX ...",Jeux de plateau,150,3,6,12,99
...,...,...,...,...,...,...,...,...,...,...,...
10544,10544,ilots,Îlots,Îlots (2010),Tout d'abord vous créez votre plateau de jeu g...,Jeux de plateau,20,2,4,12,99
10545,10545,euuromind,€uromind,€uromind (1998),Le but du jeu est de faire le tour de l'Europe...,Jeux de plateau,60,2,6,8,99
10546,10546,gokiburipoka,ごきぶりポーカー,ごきぶりポーカー (2011),Un petit jeu de bluff où il faut éviter de réc...,"Animaux|Bluff|Collection / famille|Hasard (Dé,...",25,2,6,8,99
10547,10547,siehui-shephy,シェフィ - Shephy,シェフィ - Shephy (2013),Contenu : 72 cartes 1 mouton noir22 Actions7 p...,Animaux|Combinaison|Pioche|Programmation,15,1,1,<NA>,<NA>


In [27]:
jeux_clean.to_csv('../database_cleaned/jeux_clean.csv')

In [28]:
# separate number > 10 into first digit and the remainning ones
def digit_sep(n):
    nb_dig = int(np.log10(n))
    p = np.power(10, nb_dig)
    return n // p, n % p

condition = (jeux_clean["Min number of players"] >= 12) & (jeux_clean["Max number of players"].isna())

#for v in jeux_clean.loc[condition, "Min number of players"]:
    #print(v, digit_sep(v))
#jeux_clean_2 = jeux_clean.copy()
jeux_clean.loc[condition, ["Min number of players", "Max number of players"]] = jeux_clean.loc[condition, "Min number of players"].apply(lambda v : digit_sep(v)).to_list()
jeux_clean.to_csv("../database_cleaned/jeux_clean.csv")

In [29]:
jeux_clean

,Game id,Game name website,Game name UI,Game name year,Description,Type,Game duration,Min number of players,Max number of players,Age min,Age max
0,0,fever,$ Fever,$ Fever (1990),Le jeu comprend 6 dés dont les faces représent...,Jeux de dés,20,2,6,8,99
1,1,cetera,&cetera,&cetera (2003),&cetera est une extension de Roads & Boats qui...,Jeux de plateau,<NA>,1,6,14,99
2,2,43,'43,'43 (1981),'43 simule le débarquement en Sicile des force...,Jeux de guerre,240,2,4,12,99
3,3,und-tschuss,...Und Tschüss,...Und Tschüss (1997),But du jeu Remporter les cartes les plus forte...,Jeux de cartes,30,4,6,10,99
4,4,011,011,011 (2011),"L’histoire... Nous sommes en l’an 011, au XIX ...",Jeux de plateau,150,3,6,12,99
...,...,...,...,...,...,...,...,...,...,...,...
10544,10544,ilots,Îlots,Îlots (2010),Tout d'abord vous créez votre plateau de jeu g...,Jeux de plateau,20,2,4,12,99
10545,10545,euuromind,€uromind,€uromind (1998),Le but du jeu est de faire le tour de l'Europe...,Jeux de plateau,60,2,6,8,99
10546,10546,gokiburipoka,ごきぶりポーカー,ごきぶりポーカー (2011),Un petit jeu de bluff où il faut éviter de réc...,"Animaux|Bluff|Collection / famille|Hasard (Dé,...",25,2,6,8,99
10547,10547,siehui-shephy,シェフィ - Shephy,シェフィ - Shephy (2013),Contenu : 72 cartes 1 mouton noir22 Actions7 p...,Animaux|Combinaison|Pioche|Programmation,15,1,1,<NA>,<NA>


In [30]:
jeux_clean.sort_values(by=["Min number of players"], ascending=False)[:30]
jeux_clean[(jeux_clean["Min number of players"] >= 10) & (~jeux_clean["Max number of players"].isna())]

,Game id,Game name website,Game name UI,Game name year,Description,Type,Game duration,Min number of players,Max number of players,Age min,Age max
4027,4027,hollywood-lives,Hollywood Lives,Hollywood Lives (2005),Le jeu se déroule à l'âge d'or d'Hollywood (an...,Jeux de rôles,240,10,25,<NA>,<NA>


In [31]:
from collections import Counter
categories = [cat for cats in jeux_clean["Type"][~jeux_clean["Type"].isna()].str.split('|') for cat in cats]
count_cats = Counter(categories)
count_cats

Counter({'Jeux de plateau': 3163,
         'Jeux de cartes': 1742,
         'Jeux de guerre': 444,
         'Affrontement': 427,
         'Autres': 397,
         'Hasard (Dé, Cartes, ...)': 363,
         'Casse-tête': 297,
         'Placement': 285,
         'Combinaison': 263,
         'Déplacement': 257,
         'Ambiance': 250,
         'Jeux de pions': 244,
         'Médiéval-fantastique': 239,
         'Dés': 232,
         'Guerre': 226,
         'Gestion': 212,
         'Coopération': 210,
         'Gestion de main': 204,
         'Exploration': 201,
         'Animaux': 185,
         'Jeux de figurines': 185,
         'Jeu de carte à collectionner': 184,
         'Jeux de connaissances': 180,
         'Bluff': 179,
         'Fantastique': 172,
         'Majorité': 161,
         'Jeux de dés': 160,
         'Jeu de logique': 157,
         'Prise de risque': 153,
         'Science fiction': 153,
         'Commerce': 153,
         'Collection / famille': 150,
         'Médiéval': 1

In [32]:
id_games = jeux_clean[["Game id", "Game name UI"]].groupby(by="Game name UI").count()
id_games[id_games["Game id"] > 1]

jeux_clean[jeux_clean["Game name UI"] == "Waterloo"]

,Game id,Game name website,Game name UI,Game name year,Description,Type,Game duration,Min number of players,Max number of players,Age min,Age max
10089,10089,waterloo-0,Waterloo,Waterloo (1981),Waterloo fait partie de la série International...,Jeux de guerre,240,2,3,12,99
10090,10090,waterloo-2,Waterloo,Waterloo (1996),Waterloo est la version française du jeu L'Arm...,Jeux de guerre,300,1,2,14,99
10091,10091,waterloo,Waterloo,Waterloo (2003),Idée du jeu : Waterloo : la dernière bataille ...,Jeux de guerre,60,2,3,12,99
10092,10092,waterloo-3,Waterloo,Waterloo (2009),Waterloo est le volume 4 de la Treefrog Line d...,Jeux de guerre,180,1,2,12,99


In [33]:
jeux_clean["Age min"].describe()

count      9952.0
mean     9.532255
std      3.089553
min           2.0
25%           8.0
50%          10.0
75%          12.0
max         120.0
Name: Age min, dtype: Float64

In [34]:
jeux_clean.loc[(~jeux_clean["Min number of players"].isna()) & (jeux_clean["Max number of players"].isna())].shape
for column in jeux_clean.columns:
    print(column, jeux_clean[column].isnull().sum())


Game id 0
Game name website 0
Game name UI 0
Game name year 0
Description 0
Type 1665
Game duration 1426
Min number of players 131
Max number of players 384
Age min 597
Age max 597


In [35]:
jeux_clean.sort_values(by="Game duration", ascending=False).head(30)

,Game id,Game name website,Game name UI,Game name year,Description,Type,Game duration,Min number of players,Max number of players,Age min,Age max
4270,4270,invocateurs,Invocateurs,Invocateurs (2010),"Invocateur est un jeu hybride, mélange du jeu ...",Jeux de plateau,6090120,2,4,12,99
8026,8026,russian-front,Russian Front,Russian Front (1985),Russian Front est un wargame à l'échelle strat...,Jeux de guerre,601000,1,2,12,99
8477,8477,sneaky-cards-1,Sneaky Cards,Sneaky Cards (2019),Le but du jeu est de se débarrasser de toutes ...,Ambiance|Créativité|Humour|Exploration|Prise d...,525600,1,1,12,99
2518,2518,dirty-secrets,Dirty Secrets,Dirty Secrets (2011),"Un quartier défavorisé, une épouse aigrie, un ...",Autres,240600,3,6,12,99
8559,8559,spartacus,Spartacus,Spartacus (2009),80 Avant JC. La République est au seuil d'une ...,Jeux de guerre,180300,1,2,12,99
2181,2181,d-d-conquest-of-nerath,D&D - Conquest of Nerath,D&D - Conquest of Nerath (2011),Aucune description,Affrontement|Jeux de plateau|Conquête|Médiéval...,120240,2,4,12,99
6842,6842,nuts,Nuts !,Nuts ! (2005),Nuts ! est un jeu d'escarmouche avec figurines...,Jeux de figurines,90180,1,2,10,99
2711,2711,drum-roll,Drum Roll,Drum Roll (2011),L’Europe au début des années 1900. Les joueurs...,Jeux de plateau,90120,2,4,12,99
5882,5882,lock-n-load-noville-bastogne-s-outpost,"Lock'n Load : Noville, Bastogne's outpost","Lock'n Load : Noville, Bastogne's outpost (2009)","Cette boite est une extension pour ""Lock'n Loa...",Jeux de guerre,80150,1,2,12,99
1598,1598,cave-evil,Cave Evil,Cave Evil (2011),Aucune description,NaN,60240,2,4,14,99


In [ ]:
# Aucune description check
jeux_clean[jeux_clean["Description"].str.contains(r"[a-z]*Aucune[a-z]*", regex=True)].shape[0] / jeux_clean.shape[0]

0.059626504881979334